## Experiment 5

- try diffrent size of train data
- try `RandomForestClassifier` & `XGBClassifier` & `VotingClassifier`
- without normalisation


In [13]:
import time 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn .metrics import roc_auc_score,accuracy_score,confusion_matrix,f1_score
from xgboost import XGBClassifier
import json
import numpy as np


In [40]:
def evaluate_model(model,model_name:str,train:tuple,test:tuple):
    start_train=time.time()
    model.fit(train[0],train[1])
    end_train=time.time()
    start_test=time.time()
    preds = model.predict(test[0])
    end_test=time.time()
    accuracy=accuracy_score(test[1],preds)
    f1_=f1_score(test[1],preds)
    auc=roc_auc_score(test[1],preds)
    cm=confusion_matrix(test[1],preds)
    return {"model":model_name,
           "accuracy":accuracy,
            "auc":auc,
            'f1_score':f1_,
            "cm":cm,
            'training_time(ms)':1000*(end_train-start_train),
            'testing_time(ms)':1000*(end_test-start_test)
           }

In [3]:
df = pd .read_csv('../data/cleaned_data.csv')

In [5]:
with open('../result/xgbc_best_hyperparameters.json') as f :
    xgbc_best_params=json.load(f)
with open('../result/rfc_best_hyperparameters.json') as f :
    rfc_best_params=json.load(f)
    

In [6]:
models=[
    XGBClassifier(**xgbc_best_params),
    RandomForestClassifier(**rfc_best_params),
    VotingClassifier([
        ('xgb',XGBClassifier(**xgbc_best_params)),
        ('rfc',RandomForestClassifier(**rfc_best_params))
    ])
]

In [37]:
def evaluate_models(models,df):
    result =[]
    X_train,X_test,y_train,y_test=train_test_split(df.drop('target',axis=1),df.target,test_size=0.2,random_state=42)
    test=(X_test,y_test)
    thresholds=[.1,.2,.3,.4,.5,.6,.7,.8,.9,1]
    for threshold in thresholds:
        end=int(X_train.shape[0]*threshold)
        train=(X_train[:end],y_train[:end])
        for model in models:
            rs=evaluate_model(model,model.__class__.__name__,train,test)
            rs.update({
                'threshold':threshold
            })
            result.append(rs)
        

    
    return pd.DataFrame(result)

In [41]:
result = evaluate_models(models,df)

In [42]:
result

,model,accuracy,auc,f1_score,cm,training_time(ms),testing_time(ms),threshold
0,XGBClassifier,0.793478,0.778735,0.830357,"[[53, 24], [14, 93]]",257.229567,0.000000,0.1
1,RandomForestClassifier,0.831522,0.838755,0.845771,"[[68, 9], [22, 85]]",290.236950,20.268202,0.1
2,VotingClassifier,0.831522,0.838755,0.845771,"[[68, 9], [22, 85]]",417.815685,15.472651,0.1
3,XGBClassifier,0.880435,0.873528,0.899083,"[[64, 13], [9, 98]]",90.784788,9.042740,0.2
4,RandomForestClassifier,0.896739,0.893009,0.911628,"[[67, 10], [9, 98]]",322.155714,11.203289,0.2
5,VotingClassifier,0.891304,0.890157,0.905660,"[[68, 9], [11, 96]]",420.658588,29.136419,0.2
6,XGBClassifier,0.847826,0.836388,0.873874,"[[59, 18], [10, 97]]",100.755692,0.000000,0.3
7,RandomForestClassifier,0.875000,0.867035,0.894977,"[[63, 14], [9, 98]]",349.963188,16.563177,0.3
8,VotingClassifier,0.864130,0.859510,0.883721,"[[64, 13], [12, 95]]",449.837923,33.437014,0.3
9,XGBClassifier,0.831522,0.815087,0.863436,"[[55, 22], [9, 98]]",116.143942,17.145157,0.4


In [43]:
result.to_csv('../result/exp5.csv',index=False)